In [1]:
import numpy as np
import pandas as pd
import os
import re
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import torch.optim as optim
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split

In [196]:
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2LMHeadModel, GPT2Config, Trainer, TrainingArguments, DataCollatorForLanguageModeling, get_linear_schedule_with_warmup
from evaluate import load

In [464]:
from torch.optim import Adam, SGD, RMSprop

In [330]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [338]:
from tqdm import tqdm

1.	Загрузка и подготовка корпуса данных: Загрузить текстовый корпус, оценить его объем и структуру.
2.	Очистка данных: Применить разные методы очистки данных, такие как удаление пунктуации, стоп-слов, лемматизация, и оценить их влияние.
3.	Токенизация данных: Преобразовать текстовые данные в числовые представления для последующей обработки трансформером.
4.	Создание и настройка модели: Инициализировать модель трансформера и настроить гиперпараметры для обучения.
5.	Обучение модели: Провести обучение с использованием разных оптимизаторов и сравнить результаты.
6.	Оптимизация модели: Применить различные методы управления скоростью обучения и оценить их влияние на модель.
7.	Оценка модели с использованием метрик BLEU, ROUGE и Perplexity: Измерить качество модели с использованием метрик и сравнить результаты.
8.	Интерактивное тестирование модели: Реализовать интерфейс для взаимодействия с моделью и протестировать ответы модели на различные вопросы.

In [5]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed()

# 1. Введение и подготовка данных

## уже не надо

In [36]:
conversations_path = 'data/movie_conversations.txt'
lines_path = 'data/movie_lines.txt'

In [38]:
movie_lines_list = [] 
with open(lines_path, 'r', encoding='iso-8859-1') as f:
    for line in f:
        parts = line.strip().split(' +++$+++ ')
        if len(parts) == 5:
            movie_lines_list.append(parts)

movie_lines_df = pd.DataFrame(movie_lines_list, columns=['line_id', 'user_id', 'movie_id', 'character_name', 'text'])

In [40]:
def get_line_ids(line_ids_str):
    return line_ids_str.strip('[]').replace("'", "").split(', ')

In [42]:
movie_conversations_list = []
with open(conversations_path, 'r') as f:
    for line in f:
        parts = line.strip().split(' +++$+++ ')
        if len(parts) == 4:
            movie_conversations_list.append(parts)

movie_conversations_df = pd.DataFrame(movie_conversations_list, columns=['user1_id', 'user2_id', 'movie_id', 'line_ids'])
movie_conversations_df['line_ids_list'] = movie_conversations_df['line_ids'].apply(get_line_ids)
movie_conversations_df = movie_conversations_df.drop('line_ids', axis=1)

In [68]:
dialogs = []
for index, row in movie_conversations_df.iterrows():
    line_ids = row['line_ids_list']
    dialog = movie_lines_df[movie_lines_df['line_id'].isin(line_ids)]
    if not dialog.empty:
        dialog = dialog.set_index('line_id')

        valid_line_ids = [lid for lid in line_ids if lid in dialog.index]
        if valid_line_ids:
            dialog = dialog.loc[valid_line_ids].reset_index()
            dialogs.append(dialog['text'].tolist())
        else:
            print(f"В диалоге с исходными line_ids: {line_ids} не найдено ни одной валидной реплики в movie_lines_df.")
    else:
        print(f"Для диалога с line_ids: {line_ids} не найдено соответствующих реплик в movie_lines_df.")

Для диалога с line_ids: ['L128378', 'L128379', 'L128380'] не найдено соответствующих реплик в movie_lines_df.
Для диалога с line_ids: ['L128719', 'L128720'] не найдено соответствующих реплик в movie_lines_df.
Для диалога с line_ids: ['L129148', 'L129149'] не найдено соответствующих реплик в movie_lines_df.
Для диалога с line_ids: ['L129381', 'L129382'] не найдено соответствующих реплик в movie_lines_df.
Для диалога с line_ids: ['L128978', 'L128979'] не найдено соответствующих реплик в movie_lines_df.
Для диалога с line_ids: ['L129518', 'L129519'] не найдено соответствующих реплик в movie_lines_df.
Для диалога с line_ids: ['L128708', 'L128709'] не найдено соответствующих реплик в movie_lines_df.
Для диалога с line_ids: ['L128721', 'L128722', 'L128723'] не найдено соответствующих реплик в movie_lines_df.
Для диалога с line_ids: ['L129276', 'L129277'] не найдено соответствующих реплик в movie_lines_df.
Для диалога с line_ids: ['L128991', 'L128992', 'L128993', 'L128994'] не найдено соответ

In [142]:
def create_dialogs(dialogs, sep_token="[SEP]"):
  combined_dialogs = []
  for dialog in dialogs:
      if len(dialog) < 2:
          continue  # Диалоги должны содержать как минимум две реплики
      combined_dialog = sep_token.join(dialog)
      combined_dialogs.append(combined_dialog.strip())  # .strip() убирает лишние пробелы
  return combined_dialogs

In [146]:
context_response_data = create_dialogs(dialogs, sep_token="[SEP]")

In [74]:
#ontext_response_data = create_context_response_pairs(dialogs, sep_token="[SEP]")

In [148]:
train_df = pd.DataFrame(context_response_data)

In [150]:
#train_df.to_csv('DataFrame.csv')

## Из csv

In [6]:
train_df_csv = pd.read_csv('DataFrame.csv')
train_df_csv.drop('Unnamed: 0', axis=1, inplace = True)
train_df_csv.rename(columns={'0':'context'},inplace = True)

# 2. Предобработка данных

## 2.1 Очистка данных

1. Преобразование текста в нижний регистр
2. Удаление HTML-тегов и спецсимволов
3. Удаление ссылок и упоминаний
4. Удаление знаков препинания

Решил не удалять стоп-слова, а также не делать лемматизацию и стемминг, чтобы GPT обучилась более точно и качественно. Надеюсь, это не сильно увеличит время обучения..

In [9]:
def preprocess_text(text):
    if isinstance(text, str):
        # 1. Приведение к нижнему регистру
        text = text.lower()
        # 2. Удаление HTML-тегов
        text = re.sub(r'<.*?>', '', text)
        # 3. Удаление ссылок (URL)
        text = re.sub(r'http\S+|www\S+|https\S+', '', text)
        # 3. Удаление упоминаний (@user)
        text = re.sub(r'@\w+', '', text)
        # 4. Удаление знаков пунктуации (кроме апострофа)
        text = re.sub(r'^\w\s\'', '', text)
        # Удаление лишних пробелов
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    return ''

In [10]:
#train_df['context'] = train_df['context'].apply(preprocess_text)
#train_df['response'] = train_df['response'].apply(preprocess_text)

In [11]:
train_df_csv['context'] = train_df_csv['context'].apply(preprocess_text)

## 2.2 Токенизация

In [350]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
special_tokens_dict = {'sep_token': '[sep]'}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
tokenizer.pad_token = tokenizer.eos_token

model_name = "gpt2"
config = GPT2Config.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained("gpt2")
if num_added_toks > 0:
 model.resize_token_embeddings(len(tokenizer))

In [108]:
len(tokenizer)

50258

In [263]:
model.transformer.wte.num_embeddings

50258

In [265]:
MAX_LENGTH = 256  # Выберите подходящую длину

In [267]:
def tokenize_and_prepare(example, tokenizer, max_length):
    context = example['context']
    tokenized = tokenizer(context, truncation=True, padding='max_length', max_length=max_length)
    return {
        'input_ids': tokenized['input_ids'],
        'attention_mask': tokenized['attention_mask']
    }

In [269]:
tokenized_data_list = train_df_csv.apply(lambda x: tokenize_and_prepare(x, tokenizer, MAX_LENGTH), axis=1).tolist()

In [29]:
#tokenized_data_csv = train_df_csv.apply(tokenize_and_prepare, axis=1).tolist()
#tokenized_df_csv = pd.DataFrame(tokenized_data_csv)
#tokenized_df_csv

In [354]:
print("Размер словаря токенизатора:", len(tokenizer))
print("Размер эмбеддингов модели:", model.transformer.wte.num_embeddings)

Размер словаря токенизатора: 50258
Размер эмбеддингов модели: 50258


## 2.3 Разбиение на обучающую, валидационную и тестовую выборки

In [271]:
def split_data_list(data, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    train_data, temp_data = train_test_split(data, test_size=(1 - train_ratio), random_state=42)
    val_data, test_data = train_test_split(temp_data, test_size=test_ratio / (test_ratio + val_ratio), random_state=42)
    return train_data, val_data, test_data

train_list, val_list, test_list = split_data_list(tokenized_data_list)

# 3. Обучение модели

In [356]:
class ConversationDataset(Dataset):
    def __init__(self, data_list, tokenizer):
        self.data = data_list
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        input_ids = torch.tensor(item['input_ids'], dtype=torch.long)
        attention_mask = torch.tensor(item['attention_mask'], dtype=torch.long)

        sep_token_id = self.tokenizer.convert_tokens_to_ids('[sep]')
        sep_index = (input_ids == sep_token_id).nonzero(as_tuple=True)[0]
        if len(sep_index) > 0:
            split_index = sep_index[0] + 1
            labels = input_ids.clone()
            labels[:split_index] = -100  # Маскируем контекст и [sep]
        else:
            labels = torch.full_like(input_ids, -100) # Если нет [sep], маскируем все

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

In [346]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Используется GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("GPU не обнаружен, используется CPU.")

Используется GPU: NVIDIA GeForce GTX 1660 Ti with Max-Q Design


In [394]:
train_dataset_pt = ConversationDataset(train_list, tokenizer)
val_dataset_pt = ConversationDataset(val_list, tokenizer)
test_dataset_pt = ConversationDataset(test_list, tokenizer)

In [404]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    logging_dir="./logs",
    report_to="none",
    gradient_accumulation_steps=1
)

In [406]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [496]:
def calculate_perplexity(model, tokenizer, eval_dataset, device="cuda"):
    model.eval()
    dataloader = DataLoader(eval_dataset, batch_size=4)  # Adjust batch size as needed
    total_loss = []
    model.to(device)

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Calculating Perplexity"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss.append(loss.item())

    perplexity = np.exp(np.mean(total_loss))
    return perplexity

## AdamW

In [408]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_pt,
    eval_dataset=val_dataset_pt,
    data_collator=data_collator
)

In [410]:
trainer.train()
trainer.save_model("./new_trained_gpt2")
print("Обучение завершено!")

Epoch,Training Loss,Validation Loss
1,3.301000,3.319864
2,3.170500,3.283113
3,3.027200,3.278370


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


Обучение завершено!


In [166]:
tokenizer.save_pretrained("./new_trained_gpt2")

('./trained_gpt2\\tokenizer_config.json',
 './trained_gpt2\\special_tokens_map.json',
 './trained_gpt2\\vocab.json',
 './trained_gpt2\\merges.txt',
 './trained_gpt2\\added_tokens.json')

In [411]:
model_path = "./new_trained_gpt2"
tokenizer_1 = GPT2Tokenizer.from_pretrained(model_path)
tokenizer_1.pad_token = tokenizer_1.eos_token
model_1 = GPT2LMHeadModel.from_pretrained(model_path)

In [501]:
perplexity_adamW = calculate_perplexity(model_1, tokenizer_1, val_dataset_pt, device="cuda")
perplexity_adamW

Calculating Perplexity: 100%|██████████████████████████████████████████████████████| 3114/3114 [28:41<00:00,  1.81it/s]


3445086683.9986815

In [513]:
model_1.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
model_1.to(device)
print(f"Модель загружена на: {device}")
prompt_text = "how do you feel?"
input_ids = tokenizer_1.encode(prompt_text, return_tensors="pt").to(device)

output_length = 30
temperature = 0.8
top_k = 50
top_p = 0.95
no_repeat_ngram_size = 2
do_sample = True  # Добавьте эту строку

with torch.no_grad():
    output = model_1.generate(
        input_ids=input_ids,
        max_length=output_length + len(input_ids[0]),
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        no_repeat_ngram_size=no_repeat_ngram_size,
        pad_token_id=tokenizer_1.eos_token_id,
        do_sample=do_sample,  # Включите do_sample=True здесь
    )

generated_texts = tokenizer_1.batch_decode(output, skip_special_tokens=True)

print(f"Промпт: {prompt_text}")
generated_texts

Модель загружена на: cuda
Промпт: how do you feel?


["how do you feel?i feel great. i'm...i'm just...it's your...your heart... the...the heart. it's like...it"]

## Adam with scheduler

In [198]:
from torch.optim import AdamW

In [412]:
optimizer = AdamW(model.parameters(), lr=training_args.learning_rate, weight_decay=training_args.weight_decay)

warmup_steps = int(0.1 * len(train_dataset_pt)) # Например, 10% шагов для warmup
total_steps = len(train_dataset_pt) * training_args.num_train_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

# Инициализация Trainer с оптимизатором и scheduler-ом
trainer_sch = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_pt,
    eval_dataset=val_dataset_pt,
    data_collator=data_collator,
    optimizers=(optimizer, scheduler) # Передаем кортеж из оптимизатора и scheduler-а
)

In [413]:
trainer_sch.train()

Epoch,Training Loss,Validation Loss
1,2.975000,3.355036
2,2.991100,3.355534
3,2.927300,3.358753


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=43593, training_loss=2.8820528949176367, metrics={'train_runtime': 20714.9024, 'train_samples_per_second': 8.418, 'train_steps_per_second': 2.104, 'total_flos': 2.2780615163904e+16, 'train_loss': 2.8820528949176367, 'epoch': 3.0})

In [414]:
trainer_sch.save_model("./scheduled_AdamW_trained_gpt2")
tokenizer.save_pretrained("./scheduled_AdamW_trained_gpt2")
print("Обучение завершено!")

Обучение завершено!


In [415]:
model_path = "./scheduled_AdamW_trained_gpt2"
tokenizer_2 = GPT2Tokenizer.from_pretrained(model_path)
tokenizer_2.pad_token = tokenizer_2.eos_token
model_2 = GPT2LMHeadModel.from_pretrained(model_path)

In [510]:
model_2.eval()
device = "cuda" if torch.cuda.is_available() else "cpu"
model_2.to(device)
print(f"Модель загружена на: {device}")
prompt_text = "how do you feel now?"
input_ids = tokenizer_2.encode(prompt_text, return_tensors="pt").to(device)

output_length = 30
temperature = 0.8
top_k = 50
top_p = 0.95
no_repeat_ngram_size = 2
do_sample = True  # Добавьте эту строку

with torch.no_grad():
    output = model_2.generate(
        input_ids=input_ids,
        max_length=output_length + len(input_ids[0]),
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        no_repeat_ngram_size=no_repeat_ngram_size,
        pad_token_id=tokenizer_2.eos_token_id,
        do_sample=do_sample,
    )

generated_texts = tokenizer_2.batch_decode(output, skip_special_tokens=True)

print(f"Промпт: {prompt_text}")
generated_texts

Модель загружена на: cuda
Промпт: how do you feel now?


["how do you feel now?oh, nothing. i'm fine.is that all you've got? you're a bad liar. what's this? is this a"]

In [500]:
perplexity_scheduled_AdamW = calculate_perplexity(model_2, tokenizer_2, val_dataset_pt, device="cuda")
perplexity_scheduled_AdamW

Calculating Perplexity: 100%|██████████████████████████████████████████████████████| 3114/3114 [28:47<00:00,  1.80it/s]


81552387.70238216

## SGD

In [450]:
optimizer = SGD(model.parameters(), lr=training_args.learning_rate, weight_decay=training_args.weight_decay)

# Инициализация Trainer с оптимизатором (без scheduler-а)
trainer_sgd = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_pt,
    eval_dataset=val_dataset_pt,
    data_collator=data_collator,
    optimizers=(optimizer, None) # Передаем кортеж: оптимизатор и None для scheduler-а
)

In [452]:
trainer_sgd.train()

Epoch,Training Loss,Validation Loss
1,2.364800,3.387300
2,2.923600,3.374639
3,2.980300,3.370776


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=43593, training_loss=2.809060628412021, metrics={'train_runtime': 19504.818, 'train_samples_per_second': 8.94, 'train_steps_per_second': 2.235, 'total_flos': 2.2780615163904e+16, 'train_loss': 2.809060628412021, 'epoch': 3.0})

In [453]:
trainer_sgd.save_model("./sgd_trained_gpt2")
tokenizer.save_pretrained("./sgd_trained_gpt2")
print("Обучение завершено!")

Обучение завершено!


In [454]:
model_path = "./sgd_trained_gpt2"
tokenizer_3 = GPT2Tokenizer.from_pretrained(model_path)
tokenizer_3.pad_token = tokenizer_3.eos_token
model_3 = GPT2LMHeadModel.from_pretrained(model_path)

In [498]:
perplexity_sgd = calculate_perplexity(model_3, tokenizer_3, val_dataset_pt, device="cuda")
perplexity_sgd

Calculating Perplexity: 100%|██████████████████████████████████████████████████████| 3114/3114 [33:50<00:00,  1.53it/s]


269724329.7913781

## RMSprop

In [466]:
optimizer = RMSprop(model.parameters(), lr=training_args.learning_rate, weight_decay=training_args.weight_decay)

# Инициализация Trainer с оптимизатором (без scheduler-а)
trainer_rms = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_pt,
    eval_dataset=val_dataset_pt,
    data_collator=data_collator,
    optimizers=(optimizer, None) # Передаем кортеж: оптимизатор и None для scheduler-а
)

In [468]:
trainer_rms.train()

Epoch,Training Loss,Validation Loss
1,6.514700,6.236183
2,6.203000,6.130835
3,5.939400,5.806921


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=43593, training_loss=5.93833638871744, metrics={'train_runtime': 23670.6, 'train_samples_per_second': 7.366, 'train_steps_per_second': 1.842, 'total_flos': 2.2780615163904e+16, 'train_loss': 5.93833638871744, 'epoch': 3.0})

In [469]:
trainer_sgd.save_model("./rms_trained_gpt2")
tokenizer.save_pretrained("./rms_trained_gpt2")
print("Обучение завершено!")

Обучение завершено!


In [470]:
model_path = "./rms_trained_gpt2"
tokenizer_4 = GPT2Tokenizer.from_pretrained(model_path)
tokenizer_4.pad_token = tokenizer_4.eos_token
model_4 = GPT2LMHeadModel.from_pretrained(model_path)

In [499]:
perplexity_rms = calculate_perplexity(model_4, tokenizer_4, val_dataset_pt, device="cuda")
perplexity_rms

Calculating Perplexity: 100%|██████████████████████████████████████████████████████| 3114/3114 [28:48<00:00,  1.80it/s]


156302.10956592893

BLEU и ROUGE рассчитать не получилось корректно, т.к. я использовал в качестве labels замаскированный input_ids. Из-за этого по сути на вход отдельно не поступало эталонных ответов (переучивать модель сейчас уже, к сожалению, нет времени)

По perplexity лучшие результаты были получены с использованием оптимизатора RMSprop, но в целом, при диалоге с нейронкой видно, что она дообучена на киношных диалогах. Модель отвечает классическими клише из кино, ведет себя как Джеймс Бонд или Джесси из Breaking Bad